In [1]:
import json
import pandas as pd
import numpy as np
from urllib.parse import urlparse

In [2]:
df_warc = pd.read_csv('/project/0_cleaning/output_cleaning/summary-warc.csv')
df_warc.head()

,content-type,file,ip,url,wapps
0,text/html; charset=utf-8,CC-MAIN-20171119004302-20171119024302-00601.wa...,80.248.213.170,https://www.boutique-waterair.fr/pages/retour....,"0 ['PrestaShop']\nName: wapps, dtype: object"
1,text/html; charset=utf-8,CC-MAIN-20171120032907-20171120052907-00562.wa...,107.154.162.186,http://www.manga-vf.fr/anime-gate-keepers-vf-s...,"1 ['OpenCart']\nName: wapps, dtype: object"
2,text/html; charset=UTF-8,CC-MAIN-20171124180349-20171124200349-00561.wa...,136.144.137.205,https://www.machemer-online.fr/tissu-en-coton-...,"2 ['Magento']\nName: wapps, dtype: object"
3,text/html; charset=utf-8,CC-MAIN-20171123050243-20171123070243-00112.wa...,165.231.2.190,http://www.bougezpourlivg.fr/Maillot-Formation...,"3 ['OpenCart']\nName: wapps, dtype: object"
4,text/html; charset=UTF-8,CC-MAIN-20171117223659-20171118003659-00491.wa...,178.32.102.200,http://www.kalifrais.fr/kalifrais-products/pot...,"4 ['Magento']\nName: wapps, dtype: object"


In [3]:
def list_wapps_url(wapps):
    x = wapps.replace('[',']').split(']')[1].replace(' ', '').replace("'", "").replace('"','')
    if ',' in x:
        x = x.split(',')
    else:
        x = [x]
    return x

In [4]:
wapps = []
for wapp_list_str in df_warc.loc[:,'wapps']:
    wapps_list = list_wapps_url(wapp_list_str)
    wapps.append(wapps_list)
print(wapps[:3])

[['PrestaShop'], ['OpenCart'], ['Magento']]


In [5]:
def extract_domain(url):
    url_parsed = urlparse(url)
    domain = url_parsed.netloc
    return domain

In [6]:
domains = []
for url in df_warc.loc[:,'url']:
    domain = extract_domain(url)
    domains.append(domain)
print(domains[:3])

['www.boutique-waterair.fr', 'www.manga-vf.fr', 'www.machemer-online.fr']


In [7]:
print(len(domains))
print(len(wapps))

605987
605987


# Key: WAPP, Values: Domains

In [8]:
domains_per_ecom_wapp = {}

for i in range(len(domains)):
    domain = domains[i]
    wapps_list = list(dict.fromkeys(wapps[i]))
    for wapp in wapps_list:
        if wapp in domains_per_ecom_wapp:
            if domain not in domains_per_ecom_wapp[wapp]:
                domains_per_ecom_wapp[wapp].append(domain)
        else:
             domains_per_ecom_wapp[wapp] = [domain]      

In [9]:
json_content = json.dumps(domains_per_ecom_wapp)
f = open("/project/0_cleaning/output_cleaning/domains_per_ecom_wapp.json","w")
f.write(json_content)
f.close()

# Key: Domain, Values: WAPPs

In [10]:
ecom_wapps_per_domain = {}

for i in range(len(domains)):
    domain = domains[i]
    wapps_url = list(dict.fromkeys(wapps[i]))
    if domain in ecom_wapps_per_domain:
        for w in wapps_url:
            if w not in ecom_wapps_per_domain[domain]:
                ecom_wapps_per_domain[domain].append(w)
    else:
        ecom_wapps_per_domain[domain] = wapps_url

In [11]:
json_content = json.dumps(ecom_wapps_per_domain)
f = open("/project/0_cleaning/output_cleaning/ecom_wapps_per_domain.json","w")
f.write(json_content)
f.close()